In [20]:
import pandas as pd
import numpy as np
import dash
import dash_core_components as dcc
import dash_html_components as html
import matplotlib.pyplot as plt
import plotly.plotly as py
import plotly.graph_objs as go
from dash.dependencies import Input, Output

ImportError: 
The plotly.plotly module is deprecated,
please install the chart-studio package and use the
chart_studio.plotly module instead. 


This module we'll be looking at the New York City tree census. This data was provided by a volunteer driven census in 2015, and we'll be accessing it via the socrata API. The main site for the data is [here](https://data.cityofnewyork.us/Environment/2015-Street-Tree-Census-Tree-Data/uvpi-gqnh), and on the upper right hand side you'll be able to see the link to the API.

The data is conveniently available in json format, so we should be able to just read it directly in to Pandas:

In [ ]:
url = 'https://data.cityofnewyork.us/resource/nwxe-4ae8.json'
trees = pd.read_json(url)
trees.head(10)

In [ ]:
firstfive_url = 'https://data.cityofnewyork.us/resource/nwxe-4ae8.json?$limit=5&$offset=0'
firstfive_trees = pd.read_json(firstfive_url)
firstfive_trees

You can read more about paging using the Socrata API [here](https://dev.socrata.com/docs/paging.html)

In these docs, you'll also see more advanced functions (called `SoQL`) under the "filtering and query" section. These functions should be reminding you of SQL.

Think about the shape you want your data to be in before querying it. Using `SoQL` is a good way to avoid the limits of the API. For example, using the below query I can easily obtain the count of each species of tree in the Bronx:

This behavior is very common with web APIs, and I think this is useful when thinking about building interactive data products. When in a Jupyter Notebook or RStudio, there's an expectation that (unless you're dealing with truly large datasets) the data you want can be brought in memory and manipulated.

Dash and Shiny abstract away the need to distinguish between client side and server side to make web development more accessible to data scientists. This can lead to some unintentional design mistakes if you don't think about how costly your callback functions are (for example: nothing will stop you in dash from running a costly model triggered whenever a dropdown is called.)

The goal of using the Socrata is to force you to think about where your data operations are happening, and not resort to pulling in the data and performing all operations in local memory.

----------

**NOTE**: One tip in dealing with URLs: you may need to replace spaces with `'%20'`. I personally just write out the url and then follow the string with a replace:

In [18]:
'https://api-url.com/?query with spaces'.replace(' ', '%20')

'https://api-url.com/?query%20with%20spaces'

In [21]:
#build dash app

#basic example

#dropdown example 
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

app = dash.Dash(__name__, external_stylesheets=external_stylesheets)

app.layout = html.Div([
    html.Label('Borough'),
    dcc.Dropdown(
        options=[
            {'label': 'Manhattan', 'value': 'MN'},
            {'label': u'Brooklyn', 'value': 'BK'},
            {'label': 'Bronx', 'value': 'BX'},
            {'label': 'Queens', 'value': 'QN'},
            {'label': 'Staten Island', 'value': 'SI'}
        ],
        value='BK'
    ),

], style={'columnCount': 2})

@app.callback(
    Output('barplot', 'figure'),
    [Input('Borough', 'value')])
def update_figure(selected_borough):
    filtered_df = df[df.boroname == selected_borough]
    for i in filtered_df.boroname.unique():
        df_by_boro = filtered_df[filtered_df['boroname'] == i],
        name=i
        
    return {
        dcc.Graph(
            id='pay_rate_data',
            figure={
                data: go.Bar(
                    x = df_by_boro.spc_common,
                    y = df_by_boro.health
                        ),
            }
        )
    }


if __name__ == '__main__':
    app.run_server(debug=True)

NameError: name 'Output' is not defined